In [9]:
%%capture
import sys; sys.path.append('../../..')

# Requires alpaca_trade_api backtrader matplotlib==3.2.2 plotly quantstats ta-lib (also requires talib C install)
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit
import backtrader as bt
import matplotlib as mpl
import pytz
import datetime
from functools import lru_cache

import datetime

mpl.rcParams['figure.dpi'] = 250


API_KEY: str = ""
SECRET_KEY: str = ""
for l in open('../../../paper.env'):
    if l.startswith("APCA_API_KEY_ID="):
        API_KEY = l.split("=")[1].strip()
    elif l.startswith("APCA_API_SECRET_KEY="):
        SECRET_KEY = l.split("=")[1].strip()

assert API_KEY
assert SECRET_KEY

rest_api = REST(API_KEY, SECRET_KEY, 'https://paper-api.alpaca.markets')



@lru_cache()
def get_pandas_data(symbol, timeframe, start, end):
  alpaca_data = rest_api.get_bars(symbol, timeframe, start, end, adjustment='all').df
  alpaca_data.index = alpaca_data.index.tz_convert("America/New_York")

  if timeframe != TimeFrame.Day:
    alpaca_data = alpaca_data.between_time(datetime.time(9, 30), datetime.time(16, 0), include_end=False)

  return alpaca_data


def run_backtest(strategy, symbol: str, start: datetime.date, end: datetime.date, params, cash: float):
  cerebro = bt.Cerebro(stdstats=True)
  cerebro.broker.setcash(cash)
  cerebro.addstrategy(strategy, **params)
  cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
  cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradeanalyzer')
  cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name="annualreturn")
  cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')

  data1m = get_pandas_data(symbol, TimeFrame.Minute, start, end)
  feed1 = bt.feeds.PandasData(dataname=data1m, name=f"{symbol}-1m", tz=pytz.timezone('US/Eastern'))
  cerebro.adddata(feed1)

  # NOTE: -31 days so we have room for daily indicator warmup period for indicators
  datad = get_pandas_data(symbol, TimeFrame.Day, start - datetime.timedelta(days=31), end)
  feed2 = bt.feeds.PandasData(dataname=datad, name=f"{symbol}-d", tz=pytz.timezone('US/Eastern'))
  cerebro.adddata(feed2)

  results = cerebro.run()

  cerebro.plot(iplot=True)

  strat = results[0]
  return strat

def get_roi(strategy):
  annual_returns = strategy.analyzers.annualreturn.get_analysis()
  r = 1
  for ret in annual_returns.values():
    r *= 1 + ret
  r -= 1
  return r

def get_max_drawdown(strategy):
  return strategy.analyzers.drawdown.get_analysis()['max']['drawdown']/100



In [10]:
symbol = 'NRGU'
start = datetime.date(2021, 10, 1)
end = datetime.date(2022, 5, 31)
cash = 5000

In [11]:
class MyStrategy(bt.Strategy):
    params = dict(
      atr_d_period=20,
      take_profit_levels_up=1,
      stop_loss_levels_down=1,
      level_size_atr_multiplier=1,
      verbose=True,
    )

    def __init__(self):
      self.occupations = {}
      self.atr = bt.ind.ATR(self.data1, period=self.p.atr_d_period)
      # NOTE on data1: [0] is daily bar for current day (which is a premonition), so always -1 it to get yesterday's values
    
    def log(self, *message):
      if self.p.verbose:
        print(self.data.datetime.datetime(0), *message)

    def next(self):
      now = self.data.datetime.datetime(0)
      level_size = self.atr.atr[-1] * self.p.level_size_atr_multiplier
      level_to_occupy = self.data.close[0] // level_size

      # bad data
      if level_to_occupy > 40:
        return

      if level_to_occupy in self.occupations:
        # if already entered
        if self.occupations[level_to_occupy]['size'] != 0:
          return

        # if recently attempted entry
        if self.occupations[level_to_occupy]['size'] < 1 and now - self.occupations[level_to_occupy]['time'] < datetime.timedelta(minutes=10):
          return
        
        self.log(f"evacuating {level_to_occupy} due to lack of fill")
        # we entered a while back and still have not been filled, let's exit
        self.occupations[level_to_occupy]['order'].cancel()
        del self.occupations[level_to_occupy]
      
      self.occupations[level_to_occupy] = {
        'size': 0,
        'time': now,
      }
      size = 1000 // self.data.close[0]
      orders = self.buy_bracket(size=size,
        limitprice=self.data.close[0] + (self.p.take_profit_levels_up*level_size),
        stopprice=self.data.close[0] - (self.p.stop_loss_levels_down*level_size),
        tradeid=level_to_occupy)
      self.occupations[level_to_occupy]['order'] = orders[0]

    def notify_order(self, order: bt.Order):
      if order.status not in (bt.Order.Completed, bt.Order.Partial):
        return
      
      self.occupations[order.tradeid]['size'] += order.executed.size
      if 'order' in self.occupations[order.tradeid]:
        del self.occupations[order.tradeid]['order']
      if 'time' in self.occupations[order.tradeid]:
        del self.occupations[order.tradeid]['time']
      
      if self.occupations[order.tradeid]['size'] < 1:
        del self.occupations[order.tradeid]

      self.log(self.occupations)


In [12]:
strategy = run_backtest(MyStrategy, symbol, start, end,
  {'atr_d_period': 20,
 'take_profit_levels_up': 2,
 'stop_loss_levels_down': 2,
 'level_size_atr_multiplier': 1,
 'verbose': False}
  , cash
)

# Detail

In [ ]:
annual_returns = strategy.analyzers.annualreturn.get_analysis()
analysis = strategy.analyzers.tradeanalyzer.get_analysis()
print(strategy.p.__dict__)
print(f"ROI     : {get_roi(strategy):.1%}")
print(f"Drawdown: {strategy.analyzers.drawdown.get_analysis()['max']['drawdown']/100:.1%}")
print(f"Average : {analysis['total']['total']} trades @ {analysis['pnl']['gross']['average']:.2f}")
print(f"Winners : {(analysis['won']['total'] / analysis['total']['total']):.1%} @ {analysis['won']['pnl']['average']:.2f}")
print(f"Losers  : {(analysis['lost']['total'] / analysis['total']['total']):.1%} @ {analysis['lost']['pnl']['average']:.2f}")

In [ ]:
import quantstats
portfolio_stats = strategy.analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
returns.index = returns.index.tz_convert(None)
quantstats.reports.basic(returns)

In [ ]:
from src.results import metadata, from_backtest
from src import types

transactions.index = transactions.index.tz_convert("America/New_York")
transactions['time'] = transactions.index

from_backtest.write_results("dynamic-rungs", [types.FilledOrder(intention=None, symbol=t['symbol'], quantity=t['amount'], price=t['price'], datetime=t['time'].to_pydatetime()) for t in transactions.to_dict('records')], metadata.Metadata(commit_id="", last_updated=datetime.datetime.now()))

# TODO: adjust account simulator to work with orders instead of trades (since this algo is in nearly 100% of the time)

# Optimize

In [ ]:
gridsearch = dict(
    atr_d_period=[5, 14, 20],
    take_profit_levels_up=[1, 2],
    stop_loss_levels_down=[1, 2],
    level_size_atr_multiplier=[1, 0.5, 1.5, 2],
    verbose=[False],
)

time_per = datetime.timedelta(seconds=14)

choices = 1
for l in gridsearch.values():
  choices *= len(l)

estimated_time = time_per * choices
print(f"Estimated time (@ {time_per}, {choices}): {estimated_time}")
start_time = datetime.datetime.now()
print(f"(ending at {start_time + (time_per * choices)})")
print("=" * 80)

cerebro = bt.Cerebro(stdstats=True, maxcpus=1)  # inside of VS Code editor, this does not go well
cerebro.broker.setcash(cash)
cerebro.optstrategy(MyStrategy, **gridsearch)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradeanalyzer')
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name="annualreturn")

data1m = get_pandas_data(symbol, TimeFrame.Minute, start, end)
feed1 = bt.feeds.PandasData(dataname=data1m, name=f"{symbol}-1m", tz=pytz.timezone('US/Eastern'))
cerebro.adddata(feed1)

# NOTE: -31 days so we have room for daily indicator warmup period for indicators
datad = get_pandas_data(symbol, TimeFrame.Day, start - datetime.timedelta(days=31), end)
feed2 = bt.feeds.PandasData(dataname=datad, name=f"{symbol}-d", tz=pytz.timezone('US/Eastern'))
cerebro.adddata(feed2)

runs = cerebro.run()
backtest_results = [results[0] for results in runs]


end_time = datetime.datetime.now()
print(f"Done. Took {end_time - start_time}")
print(f"time per: {(end_time - start_time) / choices}")

In [ ]:
import pandas as pd

def build_row(strategy):
  analysis = strategy.analyzers.tradeanalyzer.get_analysis()
  row = {
    'roi': get_roi(strategy),
    'drawdown': get_max_drawdown(strategy),
    'trades': analysis['total']['total'],
    'average_$': analysis['pnl']['gross']['average'],
    'win_rate': analysis['won']['total'] / analysis['total']['total'],
    'average_win_$': analysis['won']['pnl']['average'],
    'average_loss_$': analysis['lost']['pnl']['average'],
    'params': strategy.p.__dict__
  }
  row.update(strategy.p.__dict__)
  return row
results = pd.DataFrame([build_row(strategy) for strategy in sorted(backtest_results, key=get_roi, reverse=True) if strategy.analyzers.tradeanalyzer.get_analysis()['total']['total'] > 0])

In [ ]:
results.sort_values('roi').tail().params[4]

In [ ]:
results.sort_values('drawdown').head()

In [ ]:
mpl.rcParams['figure.dpi'] = 150
results.plot(x='roi', y='drawdown', kind='scatter', c=results.take_profit_levels_up.map({1: 'red', 2: 'blue'}))